In [1]:
from math import *
import numpy as np

In [2]:
def cal_p(lambda_, mu, c=1):
    p = lambda_ / mu / c
    print("p is: ", p)
    return p

In [3]:
def cal_X2_exponential(mu):
    X_2 = 2 / mu**2
    print("X^2 is: ", X_2)
    return X_2

In [4]:
def cal_W_MG1(lambda_, mu, CX=1):
    p = cal_p(lambda_, mu)
    return p * (1 + CX**2) / (2 * mu * (1 - p))

assert abs(cal_W_MG1(2, 3, 1/4) + 1/3 - 0.6875) < 0.001

p is:  0.6666666666666666


In [5]:
def cal_R_MM1(lambda_, mu):
    p = cal_p(lambda_, mu)
    return 1 / (mu * (1 - p))

assert abs(cal_R_MM1(8, 10) - 0.5) < 0.001

p is:  0.8


In [6]:
def cal_p0_MMC_without_lost(lambda_, mu, c=1):
    p = cal_p(lambda_, mu, c)
    p0 = 1 / (np.sum([(c*p)**k/factorial(k) for k in range(c)]) + (c*p)**c/(factorial(c)*(1-p)))
    print("p0 is: ", p0)
    return p0

assert abs(cal_p0_MMC_without_lost(20, 10, 3) - 1/9) < 0.001

p is:  0.6666666666666666
p0 is:  0.1111111111111111


In [7]:
def cal_pk_serie_without_lost(lambda_, mu, c, k):
    p = cal_p(lambda_, mu, c)
    p0 = cal_p0_MMC_without_lost(lambda_, mu, c)
    p_serie = [p0]
    for i in range(1, k+1):
        p_serie.append(p_serie[i-1] * c * p / i)
    print("The p serie is ", p_serie)
    return np.array(p_serie)

def cal_pk_without_lost(lambda_, mu, c, k):
    return cal_pk_serie_without_lost(lambda_, mu, c, k)[-1]

assert abs(cal_pk_without_lost(20, 10, 3, 2) - 2/9) < 0.01

p is:  0.6666666666666666
p is:  0.6666666666666666
p0 is:  0.1111111111111111
The p serie is  [0.1111111111111111, 0.2222222222222222, 0.2222222222222222]


In [8]:
def cal_wait_probability_MMC_without_lost(lambda_, mu, c):
    p_serie = cal_pk_serie_without_lost(lambda_, mu, c, c)
    return 1 - np.sum(p_serie[:-1])

assert abs(cal_wait_probability_MMC_without_lost(20, 10, 3) - 4/9) < 0.001
assert abs(cal_wait_probability_MMC_without_lost(16 / 60, 1 / 2.3, 1) * 100 - 61.33) < 0.01

p is:  0.6666666666666666
p is:  0.6666666666666666
p0 is:  0.1111111111111111
The p serie is  [0.1111111111111111, 0.2222222222222222, 0.2222222222222222, 0.14814814814814814]
p is:  0.6133333333333333
p is:  0.6133333333333333
p0 is:  0.3866666666666667
The p serie is  [0.3866666666666667, 0.23715555555555556]


In [9]:
def cal_Q_MMC(lambda_, mu, c):
    p = cal_p(lambda_, mu, c)
    p0 = cal_p0_MMC_without_lost(lambda_, mu, c)
    return ((c*p)**c * lambda_ * mu * p0) / (factorial(c-1)*(c*mu-lambda_)**2)

def cal_W_MMC(lambda_, mu, c):
    Q = cal_Q_MMC(lambda_, mu, c)
    print("Q is: ", Q)
    return Q / lambda_

def cal_R_MMC(lambda_, mu, c):
    return cal_W_MMC(lambda_, mu, c) + 1 / mu

assert abs(cal_R_MMC(16 / 60, 1 / 2.3, 1) - 5.948) < 0.001

p is:  0.6133333333333333
p is:  0.6133333333333333
p0 is:  0.3866666666666667
Q is:  0.9728735632183904


In [10]:
def cal_S(lambdas, X_2):
    S = np.sum(lambdas*X_2) / 2
    print("S is: ", S)
    return S

assert abs(cal_S(np.array([0.8, 0.2]), np.array([0.5, 2])) - 0.4) < 0.001

S is:  0.4


In [11]:
def cal_U(lambdas, mus):
    p = cal_p(lambdas, mus)
    U = np.array([np.sum(p[:i]) for i in range(p.size + 1)])
    print("U is: ", U)
    return U

def cal_W_priority_nonpreemptive_MM1(lambdas, mus):
    U = cal_U(lambdas, mus)
    X_2 = cal_X2_exponential(mus)
    S = cal_S(lambdas, X_2)
    
    W = S / ((1 - U[1:]) * (1 - U[:-1]))
    print("W is: ", W)
    return W

def cal_R_priority_nonpreemptive_MM1(lambdas, mus):
    R = cal_W_priority_nonpreemptive_MM1(lambdas, mus) + 1 / mus
    print("R is: ", R)
    return R

In [12]:
def cal_weight(lambdas):
    lambda_ = np.sum(lambdas)
    weight = lambdas / lambda_
    print("weights are :", weight)
    return weight

In [13]:
def cal_total_W_priority_nonpreemptive_MM1(lambdas, mus):
    weight = cal_weight(lambdas)
    W = cal_W_priority_nonpreemptive_MM1(lambdas, mus)
    return np.sum(weight * W)

assert abs(cal_total_W_priority_nonpreemptive_MM1(np.array([2, 11]),np.array([60/11.4, 60/1.9])) * 13 - 5.704) < 0.001

weights are : [0.15384615 0.84615385]
p is:  [0.38       0.34833333]
U is:  [0.         0.38       0.72833333]
X^2 is:  [0.0722     0.00200556]
S is:  0.08323055555555557
W is:  [0.13424283 0.49414539]


In [14]:
def cal_total_R_priority_nonpreemptive_MM1(lambdas, mus):
    weight = cal_weight(lambdas)
    R = cal_R_priority_nonpreemptive_MM1(lambdas, mus)
    return np.sum(weight * R)

assert abs(cal_total_R_priority_nonpreemptive_MM1(np.array([0.8, 0.2]), np.array([1/0.5,1])) - 1.466) < 0.001
assert abs(cal_total_R_priority_nonpreemptive_MM1(np.array([2, 11]), np.array([60/11.4,60/1.9])) * 60 - 29.688) < 0.001

weights are : [0.8 0.2]
p is:  [0.4 0.2]
U is:  [0.  0.4 0.6]
X^2 is:  [0.5 2. ]
S is:  0.4
W is:  [0.66666667 1.66666667]
R is:  [1.16666667 2.66666667]
weights are : [0.15384615 0.84615385]
p is:  [0.38       0.34833333]
U is:  [0.         0.38       0.72833333]
X^2 is:  [0.0722     0.00200556]
S is:  0.08323055555555557
W is:  [0.13424283 0.49414539]
R is:  [0.32424283 0.52581206]


In [15]:
def cal_pk_MM1m(lambda_, mu, m, k):
    p = cal_p(lambda_, mu)
    pk = p**k*(1-p)/(1-p**(m+1))
    print("p_", k, " is:", pk)
    return pk

assert abs(cal_pk_MM1m(85/60, 60/37, 4, 4) * 100 - 14.988851) < 0.001

p is:  0.8736111111111111
p_ 4  is: 0.14988851075237297


In [16]:
def cal_J_MM1m(lambda_, mu, m):
    p = cal_p(lambda_, mu)
    J = p / (1 - p)-(m + 1)*(p**(m+1))/(1-p**(m+1))
    print("J is: ", J)
    return J

assert abs(cal_J_MM1m(1/17.2, 1/15.2, 7) - 2.8615254562386037) < 0.001

p is:  0.8837209302325582
J is:  2.8615254562386045


In [17]:
def cal_p0_MMcm(lambda_, mu, c, m):
    p = cal_p(lambda_, mu, c)
    p0 = 1 / (np.sum([(c*p)**k/factorial(k) for k in range(c)]) + (1-p**(m+1-c))*lambda_**c/(factorial(c)*mu**c*(1-p)))
    print("p0 is: ", p0)
    return p0

assert abs(cal_p0_MMcm(20, 10, 3, 5) - 0.128) < 0.001

p is:  0.6666666666666666
p0 is:  0.12796208530805686


In [18]:
def cal_pk_MMcm(lambda_, mu, c, m, k):
    p = cal_p(lambda_, mu, c)
    p0 = cal_p0_MMcm(lambda_, mu, c, m)
    pk = (c*p)**k/factorial(k)*p0 if k < c else c**c*p**k/factorial(c)*p0
    print("p_", k, " is: ", pk)
    return pk

assert cal_pk_MM1m(12/60, 1/4, 1, 0) - cal_pk_MMcm(12/60, 1/4, 1, 1, 0) < 0.001
assert abs(cal_pk_MMcm(20, 10, 3, 5, 5) - 0.0758) < 0.01

p is:  0.8
p_ 0  is: 0.5555555555555556
p is:  0.8
p is:  0.8
p0 is:  0.5555555555555556
p_ 0  is:  0.5555555555555556
p is:  0.6666666666666666
p is:  0.6666666666666666
p0 is:  0.12796208530805686
p_ 5  is:  0.0758293838862559


In [19]:
def cal_J_MMcm(lambda_, mu, c, m):
    p = cal_p(lambda_, mu, c)
    p0 = cal_p0_MMcm(lambda_, mu, c, m)
    p_serie = np.array([c*p/max(k,1) if k < c else p for k in range(m+1)])
    p_serie[0] = p0
    p_serie = np.cumprod(p_serie)
    print("The p serie is ", p_serie)
    J = np.sum(p_serie * np.arange(p_serie.size))
    print("J is: ", J)
    return J

assert abs(cal_J_MMcm(20, 10, 1, 5) - cal_J_MM1m(20, 10, 5)) < 0.000001
assert abs(cal_J_MMcm(20, 10, 3, 5) - 2.1137) < 0.01

p is:  2.0
p is:  2.0
p0 is:  0.015873015873015872
The p serie is  [0.01587302 0.03174603 0.06349206 0.12698413 0.25396825 0.50793651]
J is:  4.095238095238095
p is:  2.0
J is:  4.095238095238095
p is:  0.6666666666666666
p is:  0.6666666666666666
p0 is:  0.12796208530805686
The p serie is  [0.12796209 0.25592417 0.25592417 0.17061611 0.11374408 0.07582938]
J is:  2.1137440758293837


In [20]:
def cal_R_MMcm(lambda_, mu, c, m):
    J = cal_J_MMcm(lambda_, mu, c, m)
    R = J / lambda_
    print("R is: ", R)
    return R

assert abs(cal_R_MMcm(20, 10, 3, 5) - 0.1056) < 0.01

p is:  0.6666666666666666
p is:  0.6666666666666666
p0 is:  0.12796208530805686
The p serie is  [0.12796209 0.25592417 0.25592417 0.17061611 0.11374408 0.07582938]
J is:  2.1137440758293837
R is:  0.10568720379146919


In [21]:
def cal_W_MMcm(lambda_, mu, c, m):
    R = cal_R_MMcm(lambda_, mu, c, m)
    W = R - 1/mu
    print("W is: ", W)
    return W

assert abs(cal_W_MMcm(20, 10, 3, 5) * 60 - 0.34) < 0.01

p is:  0.6666666666666666
p is:  0.6666666666666666
p0 is:  0.12796208530805686
The p serie is  [0.12796209 0.25592417 0.25592417 0.17061611 0.11374408 0.07582938]
J is:  2.1137440758293837
R is:  0.10568720379146919
W is:  0.005687203791469184


In [22]:
def cal_busy_server(lambda_, mu, c):
    p = cal_p(lambda_, mu, c)
    n = c * p
    print("The average of working server is: ", n)
    return n

In [23]:
def cal_R_MM1_preemptive(lambdas, mus):
    U = cal_U(lambdas, mus)
    X_2 = cal_X2_exponential(mus)
    left = (1 / mus) / (1 - U[:-1])
    print("left is: ", left)
    numerator = np.cumsum(lambdas*X_2 / 2)
    print("numerator is: ", numerator)
    denominator = (1 - U[1:]) * (1 - U[:-1])
    print("denominator is: ", denominator)
    right = numerator / denominator
    print("right is: ", denominator)
    R = np.array(left + right)
    print("R is: ", R)
    return R

def cal_R_total_MM1_preemptive(lambdas, mus):
    weight = cal_weight(lambdas)
    R = cal_R_MM1_preemptive(lambdas, mus)
    return np.sum(weight * R)
    
assert abs(cal_R_total_MM1_preemptive(np.array([0.5, 3, 0.2]), 1 / np.array([0.4, 0.1, 1.5])) - 0.86) < 0.1

weights are : [0.13513514 0.81081081 0.05405405]
p is:  [0.2 0.3 0.3]
U is:  [0.  0.2 0.5 0.8]
X^2 is:  [0.32 0.02 4.5 ]
left is:  [0.4   0.125 3.   ]
numerator is:  [0.08 0.11 0.56]
denominator is:  [0.8 0.4 0.1]
right is:  [0.8 0.4 0.1]
R is:  [0.5 0.4 8.6]


In [24]:
def cal_R_queueing_netowork_MM1(I, mu):
    R = 1 / (mu - I)
    print("R is: ", R)
    return R

def cal_R_total_queueing_network_MM1(I, mu, lambdas):
    lambda_ = np.sum(lambdas)
    R = cal_R_queueing_netowork_MM1(I, mu)
    return np.sum(I / lambda_ * R)

assert abs(cal_R_total_queueing_network_MM1(np.array([1000, 550, 285.5, 872.75, 549.75]), np.array([1357, 727, 375, 1192, 666]), 1550) * 60 - 0.641) < 0.001

R is:  [0.00280112 0.00564972 0.01117318 0.00313234 0.00860215]


# Problema 1

Una empresa dispone de dos impresoras y cada una atiende la demanda de un departamento. Ambas impresoras son iguales y trabajan a un ritmo promedio de 1.1 trabajos por minuto. El departamento A envía un promedio de 45 trabajos por hora y el departamento B un trabajo cada 90 segundos. Las impresoras tienen capacidad suficiente como para mantener en espera todos los trabajos que se reciban. Tanto las llegadas como el servicio se pueden considerar procesos de Poisson.

In [25]:
lambda1 = 45 / 60
lambda2 = 60 / 90
mu = 1.1
print(lambda1, lambda2, mu)

0.75 0.6666666666666666 1.1


## ¿A qué modelo se ajustan los sistemas descritos? **M/M/1**

## ¿Qué proporción de trabajos tienen que esperar para ser atendidos en cada departamento?

### Departamento A

In [26]:
cal_p(lambda1, mu)

p is:  0.6818181818181818


0.6818181818181818

### Departamento B

In [27]:
cal_p(lambda2, mu)

p is:  0.606060606060606


0.606060606060606

## ¿Cuál es el tiempo medio de respuesta en cada una de las dos alternativas?

### Departamento A:

In [28]:
cal_R_MM1(lambda1, mu)

p is:  0.6818181818181818


2.8571428571428563

### Departamento B:

In [29]:
cal_R_MM1(lambda2, mu)

p is:  0.606060606060606


2.307692307692307

La empresa modifica el sistema, unificando las dos colas en una única, de forma que cuando una de las dos impresoras se queda libre, pasa a imprimir el primer trabajo de la cola, independientemente del departamento que lo envió.

## ¿A qué modelo se ajustan los sistemas descritos? **M/M/c**

In [30]:
c = 2

In [31]:
lambda_ = lambda1 + lambda2
lambda_

1.4166666666666665

## ¿Cuál es el tiempo medio de respuesta de los trabajos de cada departamento en esta nueva situación?

## Departamento A:

In [32]:
cal_R_MMC(lambda_, mu, c)

p is:  0.6439393939393938
p is:  0.6439393939393938
p0 is:  0.21658986175115216
Q is:  0.9123369195435075


1.553093440533385

## Departamento B:

In [33]:
cal_R_MMC(lambda_, mu, c)

p is:  0.6439393939393938
p is:  0.6439393939393938
p0 is:  0.21658986175115216
Q is:  0.9123369195435075


1.553093440533385

La empresa sustituye las dos impresoras por una única impresora el doble de rápido y da más prioridad a los trabajos del departamento A. Una vez iniciada la impresión de un trabajo, no se puede interrumpir.

In [34]:
mu *= 2

## ¿Cuál es el tiempo medio de respuesta de los trabajos de cada departamento, A y B

In [35]:
lambdas = np.array([lambda1, lambda2])
mus = np.array([mu, mu])

In [36]:
R = cal_R_priority_nonpreemptive_MM1(lambdas, mus)

p is:  [0.34090909 0.3030303 ]
U is:  [0.         0.34090909 0.64393939]
X^2 is:  [0.41322314 0.41322314]
S is:  0.29269972451790627
W is:  [0.44409613 1.24724872]
R is:  [0.89864159 1.70179417]


### Departamento A:

In [37]:
R[0]

0.8986415882967606

### Departamento B:

In [38]:
R[1]

1.701794170612952

# Problema 2

Una oficina bancaria tiene dos cajeros separados y con un funcionamiento independiente. Cada cajero recibe el 50% de los clientes que requieren este servicio en la oficina bancaria. El banco se está planteando sustituir la organización anterior (que llamaremos alternativa 1) por un sistema con una cola única que sirva a los dos cajeros (el primer cliente de la cola sería atendido por el primer cajero que quede libre), que llamaremos alternativa 2. El banco recibe un promedio de 38 clientes por hora que requieren servicios en el cajero por un promedio de 1.5 minutos para realizar sus gestiones. Tanto las llegadas como el servicio se pueden considerar procesos de Poisson y en todos los casos se puede suponer que las colas tienen capacidad ilimitada.

In [39]:
lambda2 = 38 / 60
mu = 1 / 1.5
print(lambda2, mu)

0.6333333333333333 0.6666666666666666


In [40]:
lambda1 = lambda2 / 2
c = 2

## ¿Qué modelos se ajustan a cada una de las dos alternativas?

Alternativa 1: **M/M/1**

Alternativa 2: **M/M/c**

## ¿Cuál es la tasa de uso del sistema en cada una de las dos alternativas?

### Alternativa 1:

In [41]:
cal_p(lambda1, mu)

p is:  0.475


0.475

### Alternativa 2:

In [42]:
cal_p(lambda2, mu, c)

p is:  0.475


0.475

## ¿Cuál es el tiempo medio de respuesta en cada una de las dos alternativas?

### Alternativa 1:

In [43]:
cal_R_MM1(lambda1, mu)

p is:  0.475


2.857142857142857

### Alternativa 2:

In [44]:
cal_R_MMC(lambda2, mu, c)

p is:  0.475
p is:  0.475
p0 is:  0.35593220338983056
Q is:  0.2767958030669896


1.9370460048426152

### ¿Cuál es el tiempo medio de espera en cada una de las dos alternativas?

### Alternativa 1:

In [45]:
cal_W_MG1(lambda1, mu)

p is:  0.475


1.3571428571428572

### Alternativa 2:

In [46]:
cal_W_MMC(lambda2, mu, c)

p is:  0.475
p is:  0.475
p0 is:  0.35593220338983056
Q is:  0.2767958030669896


0.43704600484261513

### ¿Cuál es la probabilidad de que un cliente tenga que un esperar en cada una de las dos alternativas?

### Alternativa 1:

In [47]:
cal_wait_probability_MMC_without_lost(lambda1, mu, 1)

p is:  0.475
p is:  0.475
p0 is:  0.525
The p serie is  [0.525, 0.24937499999999999]


0.475

### Alternativa 2:

In [48]:
cal_wait_probability_MMC_without_lost(lambda2, mu, c)

p is:  0.475
p is:  0.475
p0 is:  0.35593220338983056
The p serie is  [0.35593220338983056, 0.33813559322033904, 0.16061440677966105]


0.3059322033898304

Los cajeros funcionan 8 horas al día y el banco paga una penalización de 0.1 euros por cada minuto en el que en la cola hay al menos un cliente esperando. 

## ¿Cuál es el coste diario esperado de esta penalización en la alternativa 2?

In [49]:
price = 0.1

In [50]:
(1 - np.sum(cal_pk_serie_without_lost(lambda2, mu, c, c))) * price * 60 * 8

p is:  0.475
p is:  0.475
p0 is:  0.35593220338983056
The p serie is  [0.35593220338983056, 0.33813559322033904, 0.16061440677966105]


6.9752542372881265

# Problema 3

Un sistema con dos procesadores recibe trabajos de dos tipos distintos, A y B, con una tasa de 46 y 56 trabajos a la hora, respectivamente. Cada trabajo, independientemente del tipo, tarda en procesarse una media de 50 segundos y es atendido por un único procesador. La cola de espera es suficientemente grande como para mantener en espera todos los trabajos que se reciban. Tanto las llegadas como el servicio se pueden considerar como un proceso de Poisson.

In [51]:
lambdaA = 46 / 60
lambdaB = 56 / 60
mu = 60 / 50
print(lambdaA, lambdaB, mu)

0.7666666666666667 0.9333333333333333 1.2


In [52]:
c = 2
lambda_ = lambdaA + lambdaB

## ¿A qué modelo se ajusta este sistema? M/M/c

## ¿Qué proporción de trabajos tienen que esperar para ser atendidos?

In [53]:
cal_wait_probability_MMC_without_lost(lambda_, mu, c) * 100

p is:  0.7083333333333335
p is:  0.7083333333333335
p0 is:  0.17073170731707307
The p serie is  [0.17073170731707307, 0.2418699186991869, 0.17132452574525742]


58.73983739837401

## ¿Cuál es el tiempo medio de espera en el sistema?

In [54]:
cal_W_MMC(lambda_, mu, c)

p is:  0.7083333333333335
p is:  0.7083333333333335
p0 is:  0.17073170731707307
Q is:  1.426538908246226


0.8391405342624858

Uno de los procesadores se estropea y para paliar la situación, el gestor del sistema decide duplicar la velocidad de proceso de cada trabajo y dar más prioridad a los trabajos de tipo B (aunque en ningún caso se interrumpe el proceso de un trabajo ya en ejecución).

In [55]:
mu *= 2

## ¿Cuál es el tiempo medio de espera de cada uno de los trabajos, A y B?

In [56]:
W_B, W_A = cal_W_priority_nonpreemptive_MM1(np.array([lambdaB, lambdaA]), np.array([mu]*2))

p is:  [0.38888889 0.31944444]
U is:  [0.         0.38888889 0.70833333]
X^2 is:  [0.34722222 0.34722222]
S is:  0.2951388888888889
W is:  [0.48295455 1.65584416]


### Trabajos de tipo A:

In [57]:
W_A

1.655844155844156

## Trabajos de tipo B:

In [58]:
W_B

0.4829545454545454

## Promedio de ambos tipos de trabajo:

In [59]:
cal_total_W_priority_nonpreemptive_MM1(np.array([lambdaB, lambdaA]), np.array([mu]*2))

weights are : [0.54901961 0.45098039]
p is:  [0.38888889 0.31944444]
U is:  [0.         0.38888889 0.70833333]
X^2 is:  [0.34722222 0.34722222]
S is:  0.2951388888888889
W is:  [0.48295455 1.65584416]


1.0119047619047619

# Problema 4

Un agente comercial recibe todas sus llamadas en el movil y, para no perder ninguna llamada, ha considerado adecuado desactivar el buzón de voz. De esta forma, cuando recibe una llamada y está ocupado, la llamada se pierde. Recibe un promedio de 12 llamadas a la hora, con una duración media de 4 minutos. Tanto la recepción de llamadas como el tiempo de llamada se pueden considerar procesos de Poisson.

In [60]:
lambda_ = 12 / 60
mu = 1/4
print(lambda_, mu)

0.2 0.25


In [61]:
m = 1

## Este sistema se ajusta a un modelo M/M/1 con buffer limitado

## ¿Que porcentaje de tiempo el agente está ocioso, esperando llamadas?

In [62]:
cal_pk_MM1m(lambda_, mu, m, 0) * 100

p is:  0.8
p_ 0  is: 0.5555555555555556


55.55555555555556

### ¿Que porcentaje de tiempo el agente está ocupado, atendiendo llamadas?

In [63]:
cal_pk_MM1m(lambda_, mu, m, 1) * 100

p is:  0.8
p_ 1  is: 0.44444444444444453


44.44444444444445

### ¿Qué porcentaje de llamadas se pierden? Respuesta

In [64]:
cal_pk_MM1m(lambda_, mu, m, 1) * 100

p is:  0.8
p_ 1  is: 0.44444444444444453


44.44444444444445

Su compañía de teléfono le ofrece un servicio de llamada en espera de capacidad ilimitada. El agente ha calculado que cada llamada atendida le reporta un beneficio de 10 euros. 

In [65]:
profit = 10

### ¿Cuál es el precio máximo por hora que tendría que estar dispuesto a pagar por el alquiler del servicio para que le resulte rentable contratarlo?

In [66]:
lambda_ * cal_pk_MM1m(lambda_, mu, m, 1) * 60 * profit

p is:  0.8
p_ 1  is: 0.44444444444444453


53.33333333333334

El sistema contratado, además, le permite clasificar los clientes y discriminar en función de esta clasificación. El agente ha detectado que el 20% de sus clientes son mejores que el resto y los ha clasificado como super cuquis ya que, a pesar de requerir el mismo tiempo de servicio, le proporcionan un beneficio medio de 30 euros. Por ello, ha decidido darles prioridad y si cuando termina una llamada, hay algún cliente super cuqui en espera, es atendido antes que el resto. 

In [67]:
p_super_cuquis = 0.2

### ¿Cuál es el tiempo medio de respuesta de los clientes super cuquis en cada una de las tres situaciones descritas:

#### Situación inicial:

In [68]:
1 / mu

4.0

#### Situación con servicio de llamada en espera:

In [69]:
cal_R_MM1(lambda_, mu)

p is:  0.8


20.000000000000004

### Situación con discriminación en las llamadas:

In [70]:
cal_R_priority_nonpreemptive_MM1(np.array([p_super_cuquis, 1-p_super_cuquis])*lambda_, np.array([mu]*2))[0]

p is:  [0.16 0.64]
U is:  [0.   0.16 0.8 ]
X^2 is:  [32. 32.]
S is:  3.2000000000000006
W is:  [ 3.80952381 19.04761905]
R is:  [ 7.80952381 23.04761905]


7.80952380952381

# Problema 5

Una empresa ha recogido datos de las llamadas telefónicas recibidas en los últimos años en horas punta. El tiempo entre llamadas sigue una distribución exponencial con una frecuencia media de 22 llamadas a la hora. El análisis de las conversaciones revela que la duración media de las llamadas es de 4.9 minutos. La empresa tiene 3 operadores atendiendo el teléfono.

In [71]:
lambda_ = 22 / 60
mu = 1 / 4.9
c = 3
print(lambda_, mu, c)

0.36666666666666664 0.2040816326530612 3


## Este sistema se ajusta a un modelo M/M/c

## ¿Cuál es la probabilidad de que ningún operador esté ocupado?

In [72]:
cal_p0_MMC_without_lost(lambda_, mu, c)

p is:  0.5988888888888889
p0 is:  0.1466167474773593


0.1466167474773593

## ¿Cuál es la probabilidad de que una llamada tenga que esperar por encotnrar a los operadores atendiendo otra llamada? 

In [73]:
cal_wait_probability_MMC_without_lost(lambda_, mu, c)

p is:  0.5988888888888889
p is:  0.5988888888888889
p0 is:  0.1466167474773593
The p serie is  [0.1466167474773593, 0.2634214229676556, 0.23664024496594394, 0.14172121337404867]


0.35332158458904117

## ¿Cuál es el número medio de operadores ocupados?

In [74]:
cal_busy_server(lambda_, mu, c)

p is:  0.5988888888888889
The average of working server is:  1.7966666666666669


1.7966666666666669

## ¿Cuál es el número medio de llamadas en la cola

In [75]:
cal_Q_MMC(lambda_, mu, c)

p is:  0.5988888888888889
p is:  0.5988888888888889
p0 is:  0.1466167474773593


0.5275355515055211

## ¿Cuál es el tiempo medio de espera?

In [76]:
cal_Q_MMC(lambda_, mu, c) / lambda_

p is:  0.5988888888888889
p is:  0.5988888888888889
p0 is:  0.1466167474773593


1.438733322287785

# Problema 6

Poisson con unas tasas de llegadas de 13 y 24 trabajos por hora, respectivamente. El tiempo de servicio es exponencial con una tasa de 30 y 72 trabajos por hora para los trabajos A y B, respectivamente. Se considera un modelo de prioridades expulsivas con mayor prioridad a los trabajos A

In [77]:
lambdas = np.array([13, 24])
mus = np.array([30, 72])
print(lambdas, mus)

[13 24] [30 72]


## El modelo sin prioridades sistema se ajusta a un modelo M/M/1

## ¿Cuál es el tiempo medio de respuesta de los trabajos de tipo A?

In [78]:
cal_R_MM1_preemptive(lambdas, mus)[0]

p is:  [0.43333333 0.33333333]
U is:  [0.         0.43333333 0.76666667]
X^2 is:  [0.00222222 0.0003858 ]
left is:  [0.03333333 0.0245098 ]
numerator is:  [0.01444444 0.01907407]
denominator is:  [0.56666667 0.13222222]
right is:  [0.56666667 0.13222222]
R is:  [0.05882353 0.16876751]


0.058823529411764705

## ¿Cuál es el tiempo medio de respuesta de las trabajos de tipo B?

In [79]:
cal_R_MM1_preemptive(lambdas, mus)[1]

p is:  [0.43333333 0.33333333]
U is:  [0.         0.43333333 0.76666667]
X^2 is:  [0.00222222 0.0003858 ]
left is:  [0.03333333 0.0245098 ]
numerator is:  [0.01444444 0.01907407]
denominator is:  [0.56666667 0.13222222]
right is:  [0.56666667 0.13222222]
R is:  [0.05882353 0.16876751]


0.16876750700280108

## ¿Cuál es el tiempo medio de respuesta total?

In [80]:
cal_R_total_MM1_preemptive(lambdas, mus)

weights are : [0.35135135 0.64864865]
p is:  [0.43333333 0.33333333]
U is:  [0.         0.43333333 0.76666667]
X^2 is:  [0.00222222 0.0003858 ]
left is:  [0.03333333 0.0245098 ]
numerator is:  [0.01444444 0.01907407]
denominator is:  [0.56666667 0.13222222]
right is:  [0.56666667 0.13222222]
R is:  [0.05882353 0.16876751]


0.13013854190324775

# Problema 7

Un taller de reparación de vehículos con un funcionamiento continuo (24 horas al día) recibe un promedio de 4 vehículos a la hora. Los vehículos que requieren reparaciones de motor, carrocería o ambas. Los vehículos que requieren simultáneamente a ambas reparaciones son enviados primero a reparación de motor. 

* Los vehículos siguen el siguiente flujo en la empresa: Un clasificador (S1) determina el tipo de avería: motor o carrocería o ambos y envía el vehículo al servicio adecuado. Esta tarea se realiza en un promedio de 6 minutos. Como resultado, se determina que el 10% de los trabajos no son realizables y se descartan; el 35% de los trabajos requieren sólo reparación de carrocería, el 45% sólo de motor y el 10% restante de los dos.

* El servicio de reparación de motor (S2) invierte un promedio de 20 minutos en la reparación. Una vez finalizado el trabajo, los vehículos que requieren reparación de carrocería son enviados al sistema S3, mientras que el resto son enviados a un sistema de preparación del vehículos (S5) para devolvérselo a su propietario (lavado, facturación, etc.).

* El servicio de reparación de carrocería (S3) requiere un promedio de 23 minutos en la reparación. Al final del proceso, los vehículos son enviados al sistema de secado (S4).

* El sistema de secado (S4) requiere un promedio de 11 minutos. Como resultado de este proceso, el 9% de los trabajos son reenviados al sistema de reparación de carrocería, mientras que el resto son enviados al sistema de preparación del vehículo para devolvérselo a su propietario.

* El sistema de preparación del vehículo para devolvérselo a su propietario (S5) requiere un promedio de 7 minutos.

* Se considera que todos los tiempos de proceso de los sistemas se pueden aproximar a una distribución exponencial. El proceso de llegada se puede considerar de Poisson.

Denotamos por $I_i$ la tasa de llegada al servidor i

In [81]:
lambda_ = 4
mus = 1 / np.array([6, 20, 23, 11, 7])
q12 = 0.45
q13 = 0.35
q123 = 0.1
q43 = 0.09
print(lambda_, mus)

4 [0.16666667 0.05       0.04347826 0.09090909 0.14285714]


![Problema7](problema7.jpg)

In [82]:
q23 = q123 / (q12 + q123)
q25 = q12 / (q12 + q123)
q45 = 1 - q43

In [83]:
I1 = lambda_
print("I1=lambda")
I2 = (q12 + q123) * I1
print("I2=", q12+q123, "*I1")
I3 = (q13 * I1 + q23 * I2) / (1 - q43)
print("I3=", q13, "*I1 + ", q23, "*I2 + ", q43, "*I4")
I4 = I3
print("I4=I3")
I5 = q25 * I2 + q45 * I4
print("I5=", q25, "*I2 + ", q45, "*I4")
I = np.array([I1, I2, I3, I4, I5])
print(I)

I1=lambda
I2= 0.55 *I1
I3= 0.35 *I1 +  0.18181818181818182 *I2 +  0.09 *I4
I4=I3
I5= 0.8181818181818181 *I2 +  0.91 *I4
[4.         2.2        1.97802198 1.97802198 3.6       ]


In [84]:
cal_R_total_queueing_network_MM1(I / 60, mus, np.array([lambda_ / 60]))

R is:  [10.         75.         95.13636364 17.25862069 12.06896552]


117.69200626959243

# Problema 8

Una empresa de reparación de aparatos electrónicos recibe un promedio de 17 aparatos por hora y tiene el siguiente esquema de funcionamiento:

* Los clientes son atendidos en la recepción por una persona para una inspección básica del aparato y, si procede, tomar nota de los datos y recoger el aparato. Esta operación necesita un tiempo medio de 2.1 minutos. Como resultado, se determina que el 20% tienen daños de software, el 60% de los aparatos tienen daños en el hardware y el 20% restante no son reparables, por lo que son rechazados.

* Los aparatos con daño de software son revisados por un experto en este tipo de daños que procede a un examen más detallado del aparato (para lo que necesita, en promedio, 10 minutos) y, con una descripción detallada de la reparación necesaria, pasa el aparato al taller de reparación de software.

* Los aparatos con daño de hardware son revisados directamente por otro experto en este tipo de daños que procede a un examen más detallado del aparato (para lo que necesita, en promedio, 6 minutos). El 30% de los aparatos son descartados por irreparables, y, previo aviso al cliente, se destruyen; el 40% son sustituidos por un aparato nuevo, se da el aparato por reparado y se envía al departamento de atención al cliente; el 30% restante son enviados al taller de reparación de hardware.

* El taller de software repara un promedio de 10 aparatos por hora y el taller de hardware repara un promedio de 6 aparatos por hora.

* Una vez finalizada la reparación de un aparato, es enviado al departamento de atención al cliente.

* El departamento de atención al cliente procesa el pago de la reparación y el envío del aparato al cliente. Esta operación requiere un tiempo promedio de 4 minutos.

Por tanto, se puede considerar que el sistema tiene seis servidores:

* S1 : servicio de recepción, que recibe los aparatos con una tasa λ=17 aparatos por hora y necesita un promedio de 2.1 minutos para hacer una revisión básica.

* S2 : servicio de revisión de software, que recibe los aparatos que tienen una avería de software desde el servicio de recepción. Necesita un promedio de 10 minutos para revisar detenidamente cada aparato y lo envía directamente al taller de software.

* S3 : servicio de revisión de hardware, que recibe los aparatos que tienen una avería de hadware desde el servicio de recepción. Necesita un promedio de 6 minutos para revisar detenidamente cada aparato y, según la avería, lo envía al taller de hadware, lo sustituye por uno nuevo o lo destruye por irreparable.

* S4 : taller de software, que recibe los aparatos del servicio de revisión de software ( S2 ), después de repararlo (con una tasa μ4 =10), envía el aparato al servicio de atención al cliente.

* S5 : taller de hadware, que recibe los aparatos del servicio de revisión de hadware ( S2 ), después de repararlo (con una tasa μ5 =6), envía el aparato al servicio de atención al cliente.

* S6 : servicio de atención al cliente, que recibe aparatos del servicio de revisión del hadware ( S3 ) y de los dos talleres ( S4 y S5 ). Tramita el pago de la reparación en un promedio de 4 minutos por aparato.

In [85]:
lambda_ = 17
mus = 1 / np.array([2.1, 10, 6, 1/(10/60), 1/(6/60), 4])
q12 = 0.2
q13 = 0.6
q35 = 0.3
q36 = 0.4
print(lambda_, mus)

17 [0.47619048 0.1        0.16666667 0.16666667 0.1        0.25      ]


![Problema8](problema8.jpg)

In [86]:
I1 = lambda_
print("I1=lambda")
I2 = q12 * I1
print("I2=", q12, "*I1")
I3 = q13 * I1
print("I3=", q13, "*I1")
I4 = I2
print("I4=I2")
I5 = q35 * I3
print("I5=", q35, "*I3")
I6 = q36 * I3 + I4 + I5
print("I6=", q36, "*I3 + I4 + I5")
I = np.array([I1, I2, I3, I4, I5, I6])
print(I)

I1=lambda
I2= 0.2 *I1
I3= 0.6 *I1
I4=I2
I5= 0.3 *I3
I6= 0.4 *I3 + I4 + I5
[17.    3.4  10.2   3.4   3.06 10.54]


In [87]:
cal_R_total_queueing_network_MM1(I / 60, mus, np.array([lambda_]) / 60)

R is:  [   5.18518519   23.07692308 -300.            9.09090909   20.40816327
   13.4529148 ]


-156.36697181860575